In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy.stats
import scipy.io

from scipy.stats.stats import pearsonr
from scipy.stats.stats import spearmanr
import brainiak.utils.fmrisim

%matplotlib inline

In [ ]:
def FlipSign(X):
#  check if to flip sign of a network based on the spatial map
#  input: 
#      X: column vector: spatial map
#  return:
#      assigned flip sign: 1 if don't filp; -1 if flip

    sk = scipy.stats.skew(X);
#   should flip sign
    if sk < 0:
        return -1
#   should not 
    else:
        return 1

In [ ]:
# read decomposed components

comp_path = '/hd2/research/Naturalistic/code/SANS/output/NASCAR_Rank_rst0.mat'

R = 30

# load decomposed components
with h5py.File(comp_path, 'r') as f:
    test = f['result/U']
    ref = test.ref
    U = f[ref][:]
    last_comp = U[R - 1][0]
    temporal_R = np.array(f[f[last_comp][0, 0]]).T
    subject_R = np.array(f[f[last_comp][0, 2]]).T
    spatial_R = np.array(f[f[last_comp][0, 1]]).T
    
    test = f['result/Lambda']
    ref = test.ref
    lambdas = f[ref][:]
    last_lambdas = lambdas[R - 1][0]
    lambda_arr = f[last_lambdas][0, :]
    
print(spatial_R.shape)

fmri_len, _ = temporal_R.shape
subj_num, _ = subject_R.shape
vert_num, _ = spatial_R.shape

In [ ]:
# read decomposed components from ICA
ica_comp_path = '/hd2/research/Naturalistic/data/sans/ICA/output/CanICA_decomp.mat'

# load decomposed components
f = scipy.io.loadmat(ica_comp_path)
ica_temporal = f['temporal']
ica_spatial = f['spatial']

In [ ]:
plt.rc('font', size=18)

nascar_spatial_df = pd.DataFrame(spatial_R)
corr = nascar_spatial_df.corr()
corr_np = corr.to_numpy()
mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(18, 18));
cmap = sns.diverging_palette(230, 20, as_cmap=True);
sns.heatmap(abs(corr), mask=mask, cmap=cmap, fmt=".2f", vmax=.4, vmin=-0.4, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .75});

ax.set_xlabel("Network index", weight='bold'); ax.set_ylabel("Network index", weight='bold');
plt.xticks(np.arange(R), list(np.arange(R) + 1));
plt.yticks(np.arange(R), list(np.arange(R) + 1));
plt.title("Cross-network spatial absolute correlation (BrainSync+NASCAR)", weight='bold', fontsize=23);



ica_spatial_df = pd.DataFrame(ica_spatial)
corr = ica_spatial_df.corr()
corr_np = corr.to_numpy()
mask = np.triu(np.ones_like(corr, dtype=bool))

f, ax = plt.subplots(figsize=(18, 18));
cmap = sns.diverging_palette(230, 20, as_cmap=True);
sns.heatmap(abs(corr), mask=mask, cmap=cmap, vmax=.4, fmt=".2f",vmin=-0.4, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .75});
plt.xlabel("Network index", weight='bold'); plt.ylabel("Network index", weight='bold');
plt.xticks(np.arange(R), list(np.arange(R) + 1));
plt.yticks(np.arange(R), list(np.arange(R) + 1));
plt.title("Cross-network spatial absolute correlation (spatial ICA)", weight='bold', fontsize=23);